<center><h1>Qi_Hui_HW3</h1></center>
<br>
<br>

Name: Hui Qi
<br>
Github Username: yufeifeiqiqi
<br>
USC ID: 3206742781

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re
import collections
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

Get the AReM Data Set

See part (b)

### (b) Test and Train Data

In [2]:
# https://www.geeksforgeeks.org/how-to-use-glob-function-to-find-files-recursively-in-python/
# https://stackoverflow.com/questions/6773584/how-are-glob-globs-return-values-ordered
# https://www.earthdatascience.org/courses/intro-to-earth-data-science/python-code-fundamentals/work-with-files-directories-paths-in-python/os-glob-manipulate-file-paths/
# https://stackoverflow.com/questions/7099290/how-to-ignore-hidden-files-using-os-listdir
# https://docs.python.org/3/library/re.html

def listdir_nohidden(path):
    # Get list of all files/dirs in data folder with *, sort lists
    return sorted(glob.glob(os.path.join(path,"*")))

dirPath = "../data/AReM/"
folderName = ["bending1", "bending2", "cycling", "lying", "sitting", "standing", "walking"]
columnName = ["time", "avg_rss12", "var_rss12", "avg_rss13", "var_rss13", "avg_rss23", "var_rss23"]


# store all data with key: folder name in a dictionary
datasetDict = {}

trainingSet =[]
testingSet = []

cutoff = 2
count = 0

for folder in folderName:
    datasetList = []
    path = dirPath + folder
    paths_under_onefolder = listdir_nohidden(path)
    #print(paths_under_onefolder)
    
    # \D : Matches any character which is not a decimal digit
    # (...) Matches whatever regular expression is inside the parentheses, and indicates the start and end of a group
    # m.group(1)       # The first parenthesized subgroup.
    paths_under_onefolder = sorted(paths_under_onefolder, key = lambda x: int(re.match("\D*(\d+)",x).group(1)))
    #print(paths_under_onefolder)
    
    if folder not in ["bending1", "bending2"]:
        cutoff = 3
    count = 0
    for file in paths_under_onefolder:
        # read each csv file in folder
        dataset = pd.read_csv(file, skiprows = 5, header = None, names = columnName)
        # deal with none values            
        dataset = dataset.replace(np.nan, 0)
        # put data into a dictionary value as list
        datasetList.append(dataset)
            
        count +=1
        # make less than cutoff (2 or 3) testing cut:
        if count <= cutoff:
            testingSet.append(dataset)
        else:
            trainingSet.append(dataset)
    
    # match key and value for data dictionary
    datasetDict[folder] = datasetList
datasetDict
        
    

{'bending1': [       time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23
  0         0      39.25       0.43      22.75       0.43      33.75        1.3
  1       250      39.25       0.43      23.00       0.00      33.00        0.0
  2       500      39.25       0.43      23.25       0.43      33.00        0.0
  3       750      39.50       0.50      23.00       0.71      33.00        0.0
  4      1000      39.50       0.50      24.00       0.00      33.00        0.0
  ..      ...        ...        ...        ...        ...        ...        ...
  475  118750      43.33       0.47      25.00       0.00      30.00        0.0
  476  119000      43.50       0.50      25.50       0.50      30.00        0.0
  477  119250      43.50       0.50      24.75       0.43      30.00        0.0
  478  119500      43.50       0.50      24.33       0.47      30.00        0.0
  479  119750      43.50       0.50      24.25       0.43      30.00        0.0
  
  [480 rows x 7 columns],

### (c) Feature Extraction

#### i. Research

Means, Standard deviations, Maximum, Minimum, Euclidian distance, Shapelet

#### ii. Extraction

In [3]:
# https://realpython.com/python-collections-module/
# https://docs.python.org/3/library/collections.html#collections.defaultdict

# 88 rows each row representing an instance
# each column representing a time series' single statistic (6 times series, 7 statistic each)
statCollection  = collections.defaultdict(dict)
instanceNum = 0
for folder in folderName:
    for dataset in datasetDict[folder]:
        instanceNum +=1
        # feature is the time series
        featuresNum = 0
        for column in columnName[1:]:
            featuresNum +=1
            infor = dataset[column].describe()
            #print(infor)
            statCollection[instanceNum]["min" + str(featuresNum)] = infor["min"]
            statCollection[instanceNum]["max" + str(featuresNum)] = infor["max"]
            statCollection[instanceNum]["mean" + str(featuresNum)] = infor["mean"]
            statCollection[instanceNum]["median" + str(featuresNum)] = infor["50%"]
            statCollection[instanceNum]["std" + str(featuresNum)] = infor["std"]
            statCollection[instanceNum]["first quartile" + str(featuresNum)] = infor["25%"]
            statCollection[instanceNum]["third quartile" + str(featuresNum)] = infor["75%"]
tableInfor = pd.DataFrame(statCollection).T
tableInfor

,min1,max1,mean1,median1,std1,first quartile1,third quartile1,min2,max2,mean2,...,std5,first quartile5,third quartile5,min6,max6,mean6,median6,std6,first quartile6,third quartile6
1,37.25,45.00,40.624792,40.50,1.476967,39.25,42.0000,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.43,0.582915,0.0000,1.3000
2,38.00,45.67,42.812812,42.50,1.435550,42.00,43.6700,0.0,1.22,0.372438,...,1.995255,32.0000,34.50,0.00,3.11,0.571083,0.43,0.601010,0.0000,1.3000
3,35.00,47.40,43.954500,44.33,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.00,1.79,0.493292,0.43,0.513506,0.0000,0.9400
4,33.00,47.75,42.179813,43.50,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.50,0.524317,0.0000,1.0000
5,33.00,45.75,41.678063,41.75,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.43,0.389164,0.0000,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,19.50,45.33,33.586875,34.25,4.650935,30.25,37.0000,0.0,14.67,4.576562,...,3.283983,13.7300,18.25,0.00,8.32,3.259729,3.11,1.640243,2.0500,4.3225
85,19.75,45.50,34.322750,35.25,4.752477,31.00,38.0000,0.0,13.47,4.456333,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.20,1.732727,2.1575,4.5650
86,19.50,46.00,34.546229,35.25,4.842294,31.25,37.8125,0.0,12.47,4.371958,...,2.823124,14.0000,17.75,0.00,10.00,3.338125,3.08,1.656742,2.1600,4.3350
87,23.50,46.25,34.873229,35.25,4.531720,31.75,38.2500,0.0,14.82,4.380583,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.27,1.690960,2.1700,4.5000


#### iii. Standard Deviation

In [4]:
# https://pypi.org/project/bootstrapped/
# https://github.com/facebookarchive/bootstrapped/blob/master/bootstrapped/bootstrap.py

rows = []
bs_cols = ['Column', 'Standard Deviation', '90% Bootstrapped Confidence Intervals [Lower, Upper]']
for col in tableInfor:
    #print(tableInfor[col].values)
    bsr = bs.bootstrap(np.array(tableInfor[col]), stat_func=bs_stats.std, alpha = 0.1)
    row = [col, round(bsr.value,3), (round(bsr.lower_bound, 3), round(bsr.upper_bound, 3))]
    rows.append(row)
    #print(rows)
ci_df = pd.DataFrame(rows,columns= bs_cols )
ci_df  
    

,Column,Standard Deviation,"90% Bootstrapped Confidence Intervals [Lower, Upper]"
0,min1,9.515,"(8.324, 10.828)"
1,max1,4.369,"(3.477, 5.435)"
2,mean1,5.305,"(4.764, 5.932)"
3,median1,5.409,"(4.865, 6.049)"
4,std1,1.762,"(1.586, 1.96)"
5,first quartile1,6.119,"(5.628, 6.69)"
6,third quartile1,5.110,"(4.411, 5.896)"
7,min2,0.000,"(0.0, 0.0)"
8,max2,5.034,"(4.692, 5.474)"
9,mean2,1.565,"(1.437, 1.74)"


#### iv. Select Features

I will select mean, standard deviatoin, and median. Mean can help us detect the trend in time series. Standard deviation can help us detect the seasonality and slow or fast variation. If the difference between mean and median is large, we might have some outliers.

## 2. ISLR 3.7.4

### (a) Linear Train

The training RSS for the cubic regression is lower than the training RSS from the linear regression because the cubic regression makes a tigher fit to the data. More flexible model might overfits the training data. 

### (b) Linear Test

The testing RSS for the cubic regression is higher than the training RSS from the linear regression. The cubic regression model overfits on the training data, which leads to more errors when fitting the testing data. Overfitting model might not generalize well when changing to other data. 

### (c) Not Linear Train

Cubic polynomial regression has lower training RSS than the linear regression because it is more flexible. Thus, no matter what the underlying true relationhsip is, the more flexible model will more closely follow points and have lower training RSS.

### (d) Not Linear Testing

There is not enough information to tell which testing RSS will be lower for Cubic and linear regression. This is because that we do not know the degree of linearily (i.e, "we don't know how far it is from linear"). <br>
If it is closer to linear than cubic, the linear regression has lower testing RSS than the cubic regression . if it is closer to cubic than linear, the cubic regression has lower testing RSS than linear regression. 

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 

### Reference

https://www.geeksforgeeks.org/how-to-use-glob-function-to-find-files-recursively-in-python/<br>
https://stackoverflow.com/questions/6773584/how-are-glob-globs-return-values-ordered<br>
https://www.earthdatascience.org/courses/intro-to-earth-data-science/python-code-fundamentals/work-with-files-directories-paths-in-python/os-glob-manipulate-file-paths/<br>
https://stackoverflow.com/questions/7099290/how-to-ignore-hidden-files-using-os-listdir<br>
https://realpython.com/python-collections-module/<br>
https://pypi.org/project/bootstrapped/<br>
https://github.com/facebookarchive/bootstrapped/blob/master/bootstrapped/bootstrap.py<br>
http://www.science.smith.edu/~jcrouser/SDS293/assignments/solutions-hidden/a1-solution.pdf<br>